In [ ]:
!pip3 install https://github.com/EleutherAI/lm-evaluation-harness

In [2]:
!pip3 install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.0 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from huggingface_hub import notebook_login

/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip3 install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.2 MB/s eta 0:00:0031m1.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.8 MB/s eta 0:00:00m eta 0:00:010:01

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
notebook_login()

In [9]:
!lm_eval --tasks list

2024-07-10:18:21:32,130 INFO     [__main__.py:272] Verbosity set to INFO
2024-07-10:18:21:32,530 INFO     [__init__.py:403] `group` and `group_alias` keys in tasks' configs will no longer be used in the next release of lm-eval. `tag` will be used to allow to call a collection of tasks just like `group`. `group` will be removed in order to not cause confusion with the new ConfigurableGroup which will be the offical way to create groups with addition of group-wide configuations.
2024-07-10:18:21:37,016 INFO     [__main__.py:321] Available Tasks:
 - 20_newsgroups
 - aclue
 - aclue_ancient_chinese_culture
 - aclue_ancient_literature
 - aclue_ancient_medical
 - aclue_ancient_phonetics
 - aclue_basic_ancient_chinese
 - aclue_couplet_prediction
 - aclue_homographic_character_resolution
 - aclue_named_entity_recognition
 - aclue_poetry_appreciate
 - aclue_poetry_context_prediction
 - aclue_poetry_quality_assessment
 - aclue_poetry_sentiment_analysis
 - aclue_polysemy_resolution
 - aclue_readin

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=microsoft/Phi-3-mini-4k-instruct \
    --tasks truthfulqa \
    --device cpu \
    --batch_size 6 \
    --output_path ./results \
    --log_samples


2024-07-10:22:16:51,869 INFO     [__main__.py:272] Verbosity set to INFO
2024-07-10:22:16:52,294 INFO     [__init__.py:403] `group` and `group_alias` keys in tasks' configs will no longer be used in the next release of lm-eval. `tag` will be used to allow to call a collection of tasks just like `group`. `group` will be removed in order to not cause confusion with the new ConfigurableGroup which will be the offical way to create groups with addition of group-wide configuations.
2024-07-10:22:16:57,234 INFO     [__main__.py:369] Selected Tasks: ['truthfulqa']
2024-07-10:22:16:57,235 INFO     [evaluator.py:158] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2024-07-10:22:16:57,235 INFO     [evaluator.py:195] Initializing hf model, with arguments: {'pretrained': 'microsoft/Phi-3-mini-4k-instruct'}
/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
from langchain_google_vertexai import VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.chains import QAGenerationChain
import random
from tqdm import tqdm
from langchain.output_parsers import ResponseSchema, StructuredOutputParser



In [5]:
import wandb

In [6]:

    # Initialize wandb
    wandb.init(project="test")


wandb: Currently logged in as: tl-0307 (tlzl). Use `wandb login --relogin` to force relogin


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)

In [8]:
loader = PyPDFLoader("/home/debian/Downloads/test.pdf")
qa_chunks = loader.load_and_split(text_splitter=text_splitter)
print("Number of chunks for building qa eval set:", len(qa_chunks))


Number of chunks for building qa eval set: 252


In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Joke(BaseModel):
    question: str = Field(description="question")
    answer: str = Field(description="answer to the question")

parser = JsonOutputParser(pydantic_object=Joke)


In [10]:
response_schemas = [
    ResponseSchema(name="question",description="generated question"),
    ResponseSchema(
        name="answer",
        description="answer to the generat.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)



In [11]:
templ = """You are a smart assistant designed to come up with meaninful question and answer pair. The question should be to the point and the answer should be as detailed as possible.
Given a piece of text, you must come up with a question and answer pair that can be used to evaluate a QA bot. Do not make up stuff. Stick to the text to come up with the question and answer pair.

Please come up with a question/answer pair, in the specified JSON format, for the following text:
{format_instructions}
----------------
{text}"""

PROMPT = PromptTemplate(
    template=templ,
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

In [114]:
random_chunks = []
for i in range(15):
    random_chunks.append(random.randint(5, 172)) 


In [115]:
#model = VertexAI(model_name="gemini-pro")
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="llama3")
command_chain = QAGenerationChain.from_llm(llm=model, text_splitter=text_splitter)
chain = PROMPT | model | output_parser
qa = chain.invoke(qa_chunks[0].page_content)

In [116]:
qa_pairs = []

In [117]:
for idx in tqdm(random_chunks, desc="QA Generation ..."):
    qa = chain.invoke(qa_chunks[idx].page_content)
    qa["model"] = "phi3"
    qa_pairs.append(qa)


QA Generation ...: 100%|███████████████████████████████████| 15/15 [02:26<00:00,  9.73s/it]


In [80]:
qa_pairs

[{'question': 'What are some methods that employ similar running average updates as part of their training mechanism?',
  'answer': 'Methods like self-distillation [Hinton et al., 2015], Deep Q Network in reinforcement learning [Mnih et al., 2013], Mean Teacher in semi-supervised learning [Tarvainen and Valpola, 2017], and model average used in supervised and generative modeling [Jean et al., 2014] employ similar running average updates as part of their training mechanism.',
  'model': 'phi3'},
 {'question': 'What is the time taken to train SimCLR with respect to training time on ImageNet as shown in Figure 15?',
  'answer': 'The figure shows that one can achieve better accuracy during training by increasing the amount of time spent training. Specifically, using FFCV-SSL (8GPU and resolution from 160 -> 224), SimCLR achieved high validation accuracies on ImageNet in less than two days with a single GPU or just few hours utilizing eight GPUs.',
  'model': 'phi3'},
 {'question': 'What ar

In [82]:
qa = chain.invoke(qa_chunks[0].page_content)

In [81]:
import pandas as pd
qa_phi3_df = pd.DataFrame(qa_pairs)
wandb.log({"QA Eval Pair_Phi3_1": qa_phi3_df})


In [82]:
run = wandb.init(project="test")

artifact = run.use_artifact('tlzl/test/run-m96crgz6-QAEvalPair_Phi3_1:v0', type='run_table')
artifact_dir = artifact.download()


wandb:   1 of 1 files downloaded.  


In [83]:
import json
with open(f"{artifact_dir}/QA Eval Pair_Phi3_1.table.json") as f:
    data = json.load(f)
columns = data["columns"]
data = data["data"]


In [84]:
data

[['What are some methods that employ similar running average updates as part of their training mechanism?',
  'Methods like self-distillation [Hinton et al., 2015], Deep Q Network in reinforcement learning [Mnih et al., 2013], Mean Teacher in semi-supervised learning [Tarvainen and Valpola, 2017], and model average used in supervised and generative modeling [Jean et al., 2014] employ similar running average updates as part of their training mechanism.',
  'phi3'],
 ['What is the time taken to train SimCLR with respect to training time on ImageNet as shown in Figure 15?',
  'The figure shows that one can achieve better accuracy during training by increasing the amount of time spent training. Specifically, using FFCV-SSL (8GPU and resolution from 160 -> 224), SimCLR achieved high validation accuracies on ImageNet in less than two days with a single GPU or just few hours utilizing eight GPUs.',
  'phi3'],
 ['What are some of the approaches mentioned in improving representation learning wi

In [85]:
eval_df = pd.DataFrame(columns=columns, data=data)
eval_df = eval_df.sample(frac=1).reset_index(drop=True)


In [86]:
eval_df

,question,answer,model
0,What is the connection identified by Huang et ...,Huang et al. (2021) demonstrated that there ex...,phi3
1,What did Bordes et al. [2023a] find about trai...,Bordes et al. found that by investigating the ...,phi3
2,What is the time taken to train SimCLR with re...,The figure shows that one can achieve better a...,phi3
3,What are some methods that employ similar runn...,"Methods like self-distillation [Hinton et al.,...",phi3
4,What are some of the approaches mentioned in i...,Several studies have proposed various ways to ...,phi3


In [87]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

vector_store = Chroma.from_documents(documents=qa_chunks, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),persist_directory="phi3_db")

In [88]:
retriever = vector_store.as_retriever()

In [89]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

prompt = PromptTemplate.from_template(
            """
            <s> [INST] Answer the users question based on the retrieved context. If you do not find an answer in the context, respond with i dont know[/INST] </s> 
            [INST] Question: {question} 
            Context: {context} 
            Answer: [/INST]
            """
        )


fchain = ({"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser())


In [32]:
print(fchain.invoke("How can CCA be connected to the Alternating Conditional Expectation (ACE) method?"))


 Breiman and Friedman achieved a major advance in nonlinear Canonical Correlation Analysis (CCA) by connecting the solution to equation (13) with the Alternating Conditional Expectation (ACE) method, which involves alternately optimizing conditional expectations. Painsky et al., 2020 studied this connection further and established new theoretical bounds leading to CCA refinements using ACE in nonlinear settings. SSL methods like VICReg build on the idea of maximally correlated representations but utilize different objectives, such as minimizing variance or employing contrastive losses for tasks beyond simple correlation analysis. Some algorithms use running average updates similar to those seen in self-distillation and Mean Teacher approaches within semi-supervised learning contexts. These methods evolved from the foundational CCA framework that seeks relationships between variables through their cross-covariance matrices, extending into deep neural network architectures for enhanced p

In [90]:
    predictions = []

    for idx, tmp_df in tqdm(eval_df.iterrows(), desc="Generating Answers"):
        question = tmp_df.question
        response = fchain.invoke(question)

        predictions.append({"response": response})


Generating Answers: 5it [04:17, 51.43s/it]


In [94]:
eval_df

,question,answer,model
0,What is the connection identified by Huang et ...,Huang et al. (2021) demonstrated that there ex...,phi3
1,What did Bordes et al. [2023a] find about trai...,Bordes et al. found that by investigating the ...,phi3
2,What is the time taken to train SimCLR with re...,The figure shows that one can achieve better a...,phi3
3,What are some methods that employ similar runn...,"Methods like self-distillation [Hinton et al.,...",phi3
4,What are some of the approaches mentioned in i...,Several studies have proposed various ways to ...,phi3


In [95]:
qa_pairs = eval_df.to_dict("records")


In [96]:
qa_pairs

[{'question': "What is the connection identified by Huang et al. (2021) between Barlow Twins' criterion and an upper bound of contrastive loss, suggesting about self-supervised learning methods?",
  'answer': "Huang et al. (2021) demonstrated that there exists a linkage between the objective functions used in different SSL approaches - specifically showing how the Barlow Twins' criterion can be related to an upper bound of contrastive loss, thereby suggesting connections and potential unification possibilities among these methods for self-supervised learning.",
  'model': 'phi3'},
 {'question': 'What did Bordes et al. [2023a] find about training SimCLR on ImageNet with small batch sizes using one GPU without significant performance drops?',
  'answer': 'Bordes et al. found that by investigating the impact of learning rate and optimizer settings, it is possible to effectively train SimCLR on ImageNet datasets even when employing smaller than usual batch sizes while only utilizing a sing

In [97]:
from langchain.evaluation.qa import QAEvalChain

eval_chain = QAEvalChain.from_llm(llm = model)
graded_outputs = eval_chain.evaluate(
    qa_pairs, predictions, question_key="question", prediction_key="response"
)


In [98]:
graded_outputs

[{'results': ' INCORRECT ✅\n\nThe student answer contains more information than what is provided in the true answer but does not contradict it directly. It introduces ideas about similar objectives through different mathematical frameworks which are beyond what\'s stated in the True Answer and also misinterprets by saying "aiming towards converging toward effective representation learning," whereas, according to the given information, no such inference or convergence was claimed between these methods is mentioned explicitly. The student answer should be focused solely on stating that Huang et al., (2021) demonstrated a connection without inferring broader implications about different SSL approaches and their objectives not detailed in the true response. \nGRADE: INCORRECT'},
 {'results': " GRADE: CORRECT ✅ (The student's answer does not conflict with the true answer and correctly identifies that Bordes et al.'s study demonstrates effective training of SimCLR on ImageNet using smaller b

In [101]:
    correct = 0
    for graded_output in graded_outputs:
        assert isinstance(graded_output, dict)
        if "GRADE: CORRECT" in graded_output["results"]: 
            correct+=1


In [102]:
correct

1

In [103]:
    llm_accuracy = (correct/len(graded_outputs))*100
    wandb.log({"llm_based_eval_acc": llm_accuracy}, commit=False)


In [104]:
llm_accuracy

20.0

In [105]:
from evaluate import load
squad_metric = load("squad")


In [106]:
qa_pairs[0]

{'question': "What is the connection identified by Huang et al. (2021) between Barlow Twins' criterion and an upper bound of contrastive loss, suggesting about self-supervised learning methods?",
 'answer': "Huang et al. (2021) demonstrated that there exists a linkage between the objective functions used in different SSL approaches - specifically showing how the Barlow Twins' criterion can be related to an upper bound of contrastive loss, thereby suggesting connections and potential unification possibilities among these methods for self-supervised learning.",
 'model': 'phi3'}

In [107]:
    for i, eg in enumerate(qa_pairs):
        eg["id"] = str(i)
        eg["answers"] = {"text": [eg["answer"]], "answer_start": [0]}
        predictions[i]["id"] = str(i)
        predictions[i]["prediction_text"] = predictions[i]["response"]



In [108]:
    for p in predictions:
        del p["response"]


In [109]:
    new_qa_pairs = qa_pairs.copy()
    for eg in new_qa_pairs:
        del eg["question"]
        del eg["answer"]
        del eg["model"]


In [110]:
    import numpy as np
    exact_matches = []
    f1s = []

    for qa_pair, prediction in zip(new_qa_pairs, predictions):
        result = squad_metric.compute(
            references=[qa_pair],
            predictions=[prediction],
        )
        exact_matches.append(result["exact_match"])
        f1s.append(result["f1"])

    mean_exact_match = np.mean(exact_matches)
    mean_f1 = np.mean(f1s)


In [111]:
mean_f1

26.139919509258085

In [112]:
    eval_df["Prediction"] = predictions
    eval_df["LLM Evaluating LLM"] = graded_outputs
    eval_df["Exact Match"] = exact_matches
    eval_df["F1"] = f1s

    wandb.log({"QA Eval Result Phi3": eval_df})


In [113]:
eval_df["LLM Evaluating LLM"]

0    {'results': ' INCORRECT ✅

The student answer ...
1    {'results': ' GRADE: CORRECT ✅ (The student's ...
2    {'results': ' INCORRECT

The student's answer ...
3    {'results': ' INCORRECT because the student's ...
4                         {'results': ' I don’t know'}
Name: LLM Evaluating LLM, dtype: object